# Implementing the BSBI Algorithm for Inverted Indexing

## importing libraries
Let's first import libraries we might need. we will use some libraries as typing checking, and some libraries like string just for punctuation. I use os and json libraries for reading files. And also, as it is said in Problem file, we can use nltk library for tokenizing. I implement two way for tokenizing, one using nltk and one without.

In [1]:
import json
import os
import string
from typing import List

import nltk
import re

## Utils functions
Now let's define some utils functions that are used in gamma coding:

In [2]:
def dec_to_bin(dec_num: int):
    """
    This function converts a decimal number to binary number
    """
    binary_string = bin(dec_num)[2:]
    return binary_string


def bin_to_dec(binary_string: str):
    """
    This function converts a binary number to decimal number
    """
    if not all(bit in '01' for bit in binary_string):
        raise ValueError("Input must be a binary string")

    decimal_number = int(binary_string, 2)
    return decimal_number


def dec_to_gamma(dec_num: int):
    """
    This function convert a decimal number to gamma code.
    :param dec_num: int
        The number you want to encode
    :return:
        return Gamma code of the number
    """
    bin_num = dec_to_bin(dec_num)
    n = len(bin_num)
    unary = (n) * '1'
    binary = bin_num[1:]
    gamma_code = unary + '0' + binary
    return gamma_code


def list_to_gamma_code(ls):
    """
    This function convert list of differences of numbers to gamma code using dec_to_gamma() function.
    :param ls:
        The list you want to encode
    :return:
        Gamma code of the list
    """
    curr_diff = ls[0]
    result = dec_to_gamma(curr_diff)
    for i in range(1, len(ls)):
        curr_diff = ls[i] - ls[i - 1]
        result += dec_to_gamma(curr_diff)
    return result


def gamma_code_to_list(gamma_code_list):
    """
    This function decoded a gamma code of a list. In this function, we simply loop over the gamma code, and count 1s until
    reaching a 0. Then, we decode the substring of the gamma code such that the beginning of the string is the index that
    we reach when counting 1s, here it's i, and the end of the substring which is i+counted 1s.
    :param gamma_code_list:
        The gamma code of a list
    :return:
        Return the decoded list
    """
    result = []
    i = 0
    while i < len(gamma_code_list):
        j = 0
        while gamma_code_list[i] == '1':
            j += 1
            i += 1
        dec_num = bin_to_dec('1' + gamma_code_list[i + 1: i + j])
        result.append(dec_num)
        i += j
    for i in range(1, len(result)):
        result[i] = result[i - 1] + result[i]
    return result


## Preprocessing
Let's define our preprocess functions

In [3]:
def preprocess_text(text):
    """
    Preprocesses the text by tokenizing, converting to lowercase, removing punctuation,
    and filtering out stop words.
    """
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = nltk.tokenize.word_tokenize(text)
    # Filter out stop words
    filtered_tokens = [token for token in tokens if token not in nltk.corpus.stop_words]
    return filtered_tokens


## Main classes
Now let's define our classes. I define two classes for this project. Token and Information Retrieval System. I use Token class inside of IR System.
### Algorithm
Befor going to implementation, let's summarize our algorithm and approach:
First of all, we get a list of documents_paths in format of string. We split this documents_paths with batch size, and when looping over documents, we read some of documents and make their posting list then save this posting list in disk and next we go to the next batch of documents and so on. we need to tokenize all these documents. we can split all words using whitespace. Then, we need to get rid of punctuation. This will be done using nltk. at last, it's better to lower case all words so we can index for example "another" and "Another" in same. After tokenizing, we have to create our posting_list, which is the core of this project. To do this, we should loop over all documents, then inside this loop, we loop over all the tokens that are in the current document. Then we check if the length of posting_list is zero, then we add this token as first word. else if the length of posting_list is more than 0, we find the correct index of the token in posting_list alphabetically. then we check if this token, has been already in posting_list, we just add the current document index in tokens.docs, else, we add this token in the posting_list, then add the current document index. Also we store index of occurence of the token in the relevant documents. And when all documents have been indexed and the batch posting lists are saved, we read them from disk and merge them into main posting list.
Now we create our posting list, we can easily query. for intersecting(AND) 2 words, we just need to return document's index of our 2 words that are occurred in same documents. for union(OR), we gather index of word1 with word2. for NOT, we just need to return all document's index that are not in the list of documents that contain our word. And at last, for near, we just need to find documents that either each of these word has been occurred near by at most k words on left or right. we have to cases:
 * 1)the first_word occurred before second_word.
 * 2)second_word occurred before first_word.

so we loop over a tuple (p1, p2) to cover ordered occurrence of each word. then for each word we loop over its documents, and in each document, we loop over index of that the token is occurred. then if other word(let's say second) is in documents[word1_idx:word1_idx+length], we add the current document index.

### Token class:

In [4]:
class Token:
    """
    In this class, I implement a token object, which store the valuse of a token(str), the list of documents that it is
    occurred in, and its index in them.
    ...
    Attributes:
    ----------
    word: str
        value of a token
    docs: list[int] | str
        list of documents that it is occurred in or a decoded using gamma code
    """

    def __init__(self, word: str):
        self.word: str = word
        self.docs: list | str = []

    def __str__(self):
        return self.word

    def __repr__(self):
        return self.word

### InvertedIndex model
Now let's define our main class

In [5]:
import json
import os

import nltk
import string
from typing import List

from src.preprocessing import preprocess_text
from src.utils import gamma_code_to_list, list_to_gamma_code


class Token:
    def __init__(self, word: str):
        self.word = word
        self.docs = []

    def __str__(self):
        return self.word

    def __repr__(self):
        return self.word


class InvertedIndex:
    """
    In this class, I implement an information retrieval system which can search a query among documents.
    ...
    Attributes:
    -----------
    documents: List
        list of documents in format of string.
    posting_list: List[Token]
        list of Token objects. Tokens store a string and document's indexes
    stop_word: set
        set of stop words to check when tokenizing

    Methods
    -------
    Methods defined here:
        __init__(self, documents: List):
            Constructor will set initial attributes like documents. NOTE that documents should be
            list of strings at first.
            :parameter
            ---------
            documents:List
                list of strings at first. but then chagnes to list of lists of strings
            :return
                None

        create_posting_list(self):
            calling this function, will create posting list of all occurred words cross all documents. in this function,
            we use BSBI algorithm, we read batch of documents from disk and create multiple posting_list with that
            batch size and save them on disk. In the end, we merge all these batch posting lists. Clearly, in this
            implementation we use less RAM because we don't read all documents at once.
            :parameter
                None
            :return
                None
        get_token_index(self, x):
            this function finds index of a word in posting list using binary search algorithm.
            :parameter
                x:str
                    the word you want to find its index
            :return
                int: index of the word in posting_list
        get_token(self, token):
                This function will return the token object that contains docs information. if the given token is not in the
                posting_list, it returns the spell corrected token.
                :param token:
                    token you want to fetch it from posting list
                :return:
                    return the instance of token from posting list
        get_word_docs(self, word: str):
            this simple function gets a token and will return all indexes of documents that this token is appeared in.
            :param word:
                a word that you want to search.
            :return:
                list of indexes of documents.
    """

    def __init__(self):
        """
        Constructor will set initial attributes like documents. NOTE that documents should be
            list of strings at first.
            :parameter
            ---------
            documents:List
                list of strings at first. but then changes to list of lists of strings
            :return
                None
        """
        self.document_paths: List = []
        self.posting_list: List[Token] = []
        self.stop_words: set = set(nltk.corpus.stopwords.words('english') + list(string.punctuation))

    def get_document_paths(self, path=os.getcwd()):
        for filename in sorted(os.listdir(path)):
            filepath = os.path.join(path, filename)
            if filepath.endswith('.txt'):
                # Use encoding cp1252 for test cases if an error raised
                self.document_paths.append(filepath)
        return self.document_paths

    def create_posting_list(self, batch_size=1):
        """
        calling this function, will create posting list of all occurred words cross all documents. in this function, we
        loop over all documents_path and read numbers of them(with batch size), then inside this loop, we loop over all
        the tokens that are in the current document. then we check if the length of posting_list is zero, then we add
        this token as first word. else if the length of posting_list is more than 0, we find the correct index of the
        token in posting_list alphabetically. then we check if this token, has been already in posting_list, we just add
        the current document index in tokens.docs, else, we add this token in the posting_list, then add the current
        document index. And when all documents have been indexed and the batch posting lists are saved, we read them from
        disk and merge them into main posting list.
            :parameter
                None
            :return
                None
        :return:
        """
        temp_posting_list = []
        for doc_idx in range(len(self.document_paths)):
            if doc_idx % batch_size == 0:
                temp_posting_list = []
            doc_path = self.document_paths[doc_idx]
            doc = open(doc_path, encoding='cp1252').read()
            doc = preprocess_text(doc)
            batch_idx = doc_idx // batch_size
            for token in doc:
                if temp_posting_list == 0:
                    temp_posting_list.append(Token(token))
                    temp_posting_list[0].docs.append(doc_idx)
                    continue
                i = 0
                while i < len(temp_posting_list) and token > temp_posting_list[i].word:
                    i += 1
                if i == len(temp_posting_list):
                    temp_posting_list.append(Token(token))
                    # self.posting_list[i].post_idx.append(post_idx)
                elif token != temp_posting_list[i].word:
                    temp_posting_list.insert(i, Token(token))

                if doc_idx not in temp_posting_list[i].docs:
                    temp_posting_list[i].docs.append(doc_idx)

            if (doc_idx + 1) % batch_size == 0:
                self.posting_list_to_json(temp_posting_list,
                                          path=os.path.join(os.getcwd(), f"posting_lists/{batch_idx}.json"))
        self.posting_list_to_json(temp_posting_list,
                                  path=os.path.join(os.getcwd(), f"posting_lists/{batch_idx}.json"))

        path = os.path.join(os.getcwd(), 'posting_lists')
        batch_posting_lists = []
        for filename in sorted(os.listdir(path)):
            filepath = os.path.join(path, filename)
            if filepath.endswith('.json'):
                # Use encoding cp1252 for test cases if an error raised
                batch_posting_lists.append(filepath)
        curr_merge_posting_list = self.json_to_posting_list(batch_posting_lists[0])
        for i in range(len(batch_posting_lists) - 1):
            temp_posting_list = self.json_to_posting_list(batch_posting_lists[i + 1])
            curr_merge_posting_list = self.merge_two_posting_list(curr_merge_posting_list,
                                                                  temp_posting_list)
        self.posting_list = curr_merge_posting_list
        # for token in self.posting_list:
        #     token.docs = list_to_gamma_code(token.docs)

    def posting_list_to_json(self, posting_list, path):
        json_posting_list = {}
        for token in posting_list:
            json_posting_list[token.word] = token.docs
        with open(path, "w") as outfile:
            json.dump(json_posting_list, outfile)
        return json_posting_list

    def json_to_posting_list(self, path):
        with open(path, 'r') as openfile:
            json_object = json.load(openfile)
            posting_list = []
            for word, docs in json_object.items():
                token = Token(word)
                token.docs = docs
                posting_list.append(token)
            return posting_list

    def merge_two_posting_list(self, posting_list_1: List[Token], posting_list_2: List[Token]):
        """
        In this function we merge two posting list. If posting lists contain tokens that their docs attribute is a str,
        we should first convert their docs to list of numbers, then merge them and at last, we convert their docs to
        gamma code.
        :param posting_list_1: First posting list we want to merge
        :param posting_list_2: Second posting list we want to merge
        :return:
            returns a merged posting list of first and second posting list.
        """
        merged_posting_list = posting_list_1
        if type(posting_list_1[0].docs) != list:
            for token in posting_list_1:
                token.docs = gamma_code_to_list(token.docs)
        if type(posting_list_2[0].docs) != list:
            for token in posting_list_2:
                token.docs = gamma_code_to_list(token.docs)
        for token in posting_list_2:

            low = 0
            high = len(merged_posting_list) - 1
            idx = 0
            while low <= high:
                idx = (high + low) // 2
                if merged_posting_list[idx].word < token.word:
                    low = idx + 1
                elif merged_posting_list[idx].word > token.word:
                    high = idx - 1
                else:
                    break
            if token.word == merged_posting_list[idx].word:
                for doc in token.docs:
                    if doc not in merged_posting_list[idx].docs:
                        merged_posting_list[idx].docs.append(doc)
            else:
                merged_posting_list.insert(idx, token)
        if type(posting_list_2[0].docs) == list:
            for token in posting_list_1:
                token.docs = list_to_gamma_code(token.docs)
        return merged_posting_list

    def get_token_index(self, x, posting_list=None):
        """
        this function find index of a word in posting list using binary search algorithm.
            :parameter
                x:str
                    the word you want to find its index
            :return
                int: index of the word in posting_list
        """
        if not posting_list:
            posting_list = self.posting_list
        low = 0
        high = len(posting_list) - 1
        mid = 0
        while low <= high:
            mid = (high + low) // 2
            if posting_list[mid].word < x:
                low = mid + 1
            elif posting_list[mid].word > x:
                high = mid - 1
            else:
                return mid
        return -1

    def get_token(self, token):
        """
        This function will return the token object that contains docs informations. if the given token is not in the
        posting_list, it return the spell corrected token.
        :param token:
            token you want to fetch it from posting list
        :return:
            return the instance of token from posting list
        """
        p = self.get_token_index(token)
        if p == -1:
            null_token = Token('token')
            null_token.docs = []
            return null_token
        return self.posting_list[p]


nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     UNEXPECTED_EOF_WHILE_READING] EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     UNEXPECTED_EOF_WHILE_READING] EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1006)>


False

In this class, I implement an information retrieval system which can search a query among documents.
    The attributes are:
   * documents: List
        list of documents in format of string.
   * posting_list: List[Token]
        list of Token objects. Tokens store a string and document's indexes
   * stop_word: set
        set of stop words to check when tokenizing

### Methods
Methods defined here:

   * __init__(self, documents: List, case_sensitive=False):
        Constructor will set initial attributes like documents and case_sensitive. NOTE that documents should be
        list of strings at first.
        and parameters are:
        * documents:List
            list of strings at first. but then chagnes to list of lists of strings
        and it returns nothing

   * tokenize_document(self, document, use_nltk=False):
        in this function, we tokenize a documents. if use_nltk=True, we tokenize using nltk library. if case_sensitive has been set to True, we lower all tokens.
        parameters are
        * use_nltk: bool, Optional
        * use_nltk: bool, Optional
        and it returns nothing

   * create_posting_list(self):
        calling this function, will create posting list of all occurred words cross all documents. in this function,
            we use BSBI algorithm, we read batch of a documents from disk and create multiple posting_list with that
            batch size and save them on disk. At the end we merge all these batch posting lists. Clearly, in this
            implementation we use less RAM, because we don't read all documents at once.

   * merge_two_posting_list(self, posting_list_1: List[Token], posting_list_2: List[Token]):
            In this function we merge two posting list. If posting lists contain tokens that their docs attribute is a str,
            we should first convert their docs to list of numbers, then merge them and at last, we convert their docs to
            gamma code.
            :param posting_list_1: First posting list we want to merge
            :param posting_list_2: Second posting list we want to merge
            :return:
                returns a merged posting list of first and second posting list.

   * get_token_index(self, x):
        this function find index of a word in posting list using binary search algorithm.
        parameters are
        * x:str
                the word you want to find its index
        and it returns:
        * int: index of the word in posting_list


## Querying
At last we complete our IR system by implementing a QueryProcessor class. This class will use the InvertedIndex class we defined before.

In [6]:
class QueryProcessor:
    """
    In this class, I implement an information retrieval system which can search a query among documents.
    ...
    Attributes:
    -----------
    indexing_model: InvertedIndex
        an instance of InvertedIndex class which has been created by indexing documents.
    Methods
    -------
    Methods defined here:
        get_word_docs(self, word: str):
                this simple function gets a token and will return all index of documents that this token is appeared in.
                :param word:
                    a word that you want to search.
                :return:
                    list of indexes of documents.
        intersect(self, first_word, second_word):
                this function get two words, and find documents that both of these word has been occurred.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            union(self, first_word, second_word):
                this function get two words, and find documents that either each of these word has been occurred.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            not_in(self, word):
                this function get one word, and find documents that this word has been not occurred.
                :parameter
                word: str
                    the word you want to search
                :return
                    list of indexes of documents.
            near(self, first_word, second_word, length):
                this function get two words, and find documents that either each of these word has been occurred near by at most 3 words on left or right.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
            search(self, query):
                this function get a query and recognize what kind of query is; then search the query.
                :parameter
                    query: str
                        the query that user wants to search
                :return
                    print list of indexes of documents in a pretty way.
    """
    def __init__(self, indexing_model):
        self.indexing_model = indexing_model

    def get_word_docs(self, word: str):
        """
        This simple function gets a token and will return all index of documents that this token is appeared in. It first
        gets the gamma-coded lists, and then convert it to a list of numbers.
        :param word:
            a word that you want to search.
        :return:
            list of indexes of documents.
        """
        t = self.indexing_model.get_token(word)
        result = set(gamma_code_to_list(t.docs))
        return result

    def intersect(self, first_word, second_word):
        """
        this function get two words, and find documents that both of these word has been occurred. we first find index of
        each word, then in posting list, we find indexes of documents that this word is occurred in. at last, we add the
        index of documents in result, if it exists in both token.docs.
            :parameter
            first_word: str
                first word you want to search
            second_word: str
                second word you want to search
            :return
                list of indexes of documents.
        """
        t1 = self.get_word_docs(first_word)
        t2 = self.get_word_docs(second_word)
        if len(t1) > len(t2):
            t1, t2 = t2, t1
        result = []
        for doc1 in t1:
            for doc2 in t2:
                if doc1 == doc2:
                    result.append(doc1)
        return result

    def union(self, first_word, second_word):
        """
            this function get two words, and find documents that either each of these word has been occurred. we first
            find index of each word, then in posting list, we find indexes of documents that each word is occurred in.
            at last, we add all found index of documents in result.
                :parameter
                first_word: str
                    first word you want to search
                second_word: str
                    second word you want to search
                :return
                    list of indexes of documents.
        """
        t1 = self.get_word_docs(first_word)
        t2 = self.get_word_docs(second_word)
        if len(t1) > len(t2):
            t1, t2 = t2, t1
        result = set()
        for doc1 in t1:
            result.add(doc1)
        for doc2 in t2:
            result.add(doc2)
        return list(result)

    def not_in(self, word):
        """
        not_in(self, word):
            this function get one word, and find documents that this word has been not occurred. we loop over all docs of
            that this token is in and store it in p1_docs. then we return index of documents that are not in hat p1_docs.
            :parameter
            word: str
                the word you want to search
            :return
                list of indexes of documents.
        """
        t = self.get_word_docs(word)
        t_docs = []
        for doc in t:
            t_docs.append(doc)
        result = set()
        for idx in range(len(self.indexing_model.document_paths)):
            if idx not in t_docs:
                result.add(idx)
        return list(result)

    def search(self, query):
        query_parts = query.lower().split()
        if 'and' in query_parts:
            return self.intersect(query_parts[0], query_parts[2])
        elif 'or' in query_parts:
            return self.union(query_parts[0], query_parts[2])
        elif 'not' in query_parts:
            return self.not_in(query_parts[1])
        elif 'near' in query:
            distance = int(query_parts[1].split('/')[1])
            return self.near(query_parts[0], query_parts[2], distance)
        else:
            return list(self.get_word_docs(query_parts[0]))

### Build our system
Now let's build our system and call initial functions:

In [7]:
# Initialize the Information Retrieval System with the preprocessed documents
inverted_index = InvertedIndex()
# Get documents' path
dataset_path = '../dataset/raw'
inverted_index.get_document_paths(path=dataset_path)
# Tokenize documents
inverted_index.create_posting_list(batch_size=5)

In [8]:
print('**** ir_system documents:\n', inverted_index.document_paths)

**** ir_system documents:
 ['../dataset/raw/A Festival of Books.txt', '../dataset/raw/A Murder-Suicide.txt', '../dataset/raw/Better To Be Unlucky.txt', '../dataset/raw/Cloning Pets.txt', '../dataset/raw/Crazy Housing Prices.txt', '../dataset/raw/Food Fight Erupted in Prison.txt', '../dataset/raw/Freeway Chase Ends at Newsstand.txt', '../dataset/raw/Gasoline Prices Hit Record High.txt', '../dataset/raw/Happy and Unhappy Renters.txt', '../dataset/raw/Jerry Decided To Buy a Gun.txt', '../dataset/raw/Man Injured at Fast Food Place.txt', '../dataset/raw/Pulling Out Nine Tons of Trash.txt', '../dataset/raw/Rentals at the Oceanside Community.txt', '../dataset/raw/Sara Went Shopping.txt', '../dataset/raw/Trees Are a Threat.txt']


In [9]:
print('**** ir_system posting list:\n', inverted_index.posting_list)

**** ir_system posting list:
 [1, 10, 1000, 1000x, 100x, 10x, 11, 12, 110000, 120, 120000, 15, 150, 15percent, 1992, 1993, 1995, 1x, 20, 2000, 200, 209, 20000, 222, 214, 24yearold, 230000, 25, 25000, 25000we, 280, 3, 2995, 3037, 30, 350, 38, 300, 4000, 3995, 4000residents, 5, 50, 500, 50000, 500x, 50th, 50x, 510000it, 60, 65yearold, 6000, 7, 70000, 70yearold, 75, 74yearold, 75000, 76, 79yearold, 87, 9, 90, 900, 99, 9000, able, abuse, accidentally, acres, according, actually, administration, afghan, agency, ago, alan, aid, allotments, allow, allen, allowing, almost, along, alone, already, also, altadena, although, always, ambulance, american, ammunition, among, amount, amounts, amputated, angelenos, angeles, anniversary, another, antennas, annual, apartment, anyone, anything, apartments, apparent, appears, apply, appliances, approved, april, area, arcadia, arizona, arrived, around, asked, asking, assisted, attendance, attract, attitudes, audience, auto, authors, autograph, availablethis

Posting list with their index:

In [10]:
for token in inverted_index.posting_list:
    print(f"{token.word}: doc_list: {token.docs}")

1: doc_list: 111011
10: doc_list: 1011100011011100110010
1000: doc_list: 1100111011111001
1000x: doc_list: 1100
100x: doc_list: 1100
10x: doc_list: 1100
11: doc_list: 11110110
12: doc_list: 111011
110000: doc_list: 1100
120: doc_list: 1100
120000: doc_list: 1100
15: doc_list: 11110100
150: doc_list: 1011110010
15percent: doc_list: 11110100
1992: doc_list: 11110101
1993: doc_list: 111000
1995: doc_list: 11110101
1x: doc_list: 1100
20: doc_list: 10111010
2000: doc_list: 11110100
200: doc_list: 11110100
209: doc_list: 111011
20000: doc_list: 111000
222: doc_list: 111011
214: doc_list: 111011
24yearold: doc_list: 111010
230000: doc_list: 111000
25: doc_list: 1100
25000: doc_list: 1101
25000we: doc_list: 1101
280: doc_list: 10
3: doc_list: 11110110
2995: doc_list: 11110101
3037: doc_list: 11110101
30: doc_list: 1110111100111000
350: doc_list: 11110101
38: doc_list: 11110001
300: doc_list: 11110001
4000: doc_list: 11110110
3995: doc_list: 11110101
4000residents: doc_list: 11110110
5: doc_lis

### Testing
Now we can test and query to our system. Note that the documents are 0-based indexing:

In [11]:
ir_system = QueryProcessor(inverted_index)
# # Now we test our sample
print(ir_system.search('another'))
print(ir_system.search('back or another'))
print(ir_system.search('back'))
print(ir_system.search('young'))
print(ir_system.search('not back'))


[]
[11, 13, 14]
[11, 13, 14]
[10, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12]
